In [1]:
import os
from dotenv import load_dotenv
from pyathena import connect
from pyathena.pandas.util import as_pandas
from pyathena.arrow.cursor import ArrowCursor
import pyarrow as pa
import pyarrow.parquet as pq
import polars as pl

load_dotenv()
aws_access_key_id = os.getenv("AWS_ACCESS_KEY_ID")
aws_secret_access_key = os.getenv("AWS_SECRET_ACCESS_KEY")
region_name = os.getenv("AWS_REGION")
s3_staging_dir = os.getenv("S3_STAGING_DIR")

conn = connect(
    aws_access_key_id=aws_access_key_id,
    aws_secret_access_key=aws_secret_access_key,
    s3_staging_dir=s3_staging_dir,
    region_name=region_name)

In [4]:
def main():
    try:
        with conn.cursor() as cursor:
            cursor.execute("drop table if exists prevencao_a_fraude.vendas_cambraia;")
            print("Tabela deletada")

            with open("vendas.sql", 'r') as file:
                query_venda = file.read()
            print("Arquivo de consulta lido")
            cursor.execute(query_venda)
            print("Tabela criada")
            
            # Executar consulta final
            df_venda = as_pandas(cursor.execute('''
            select * from prevencao_a_fraude.vendas_cambraia CV
            order by CV. FILIAL, CV.DATA_VDA, CV.COO asc'''))
            print("Query de vendas finalizada!")
            print("Número de linhas:\t", len(df_venda))

            pq.write_table(df_venda, 'vendas.parquet')
            print("Parquet criado!")

    except Exception as e:
        print(f"Erro geral: {e}")
    finally:
        conn.close()
    return df_venda

df_venda = main()

Tabela deletada
Arquivo de consulta lido
Tabela criada
